In [84]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [85]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

url = 'https://www.bilbasen.dk/brugt/bil?includeengroscvr=true&includeleasing=false'

page = requests.get(url, headers=headers)
print(page)
# print(page.text)
soup = BeautifulSoup(page.text, 'html.parser')

<Response [200]>


In [86]:
# get links to all cars on page

def getCarLinks(tree):
    l = [a['href'] for a in tree.select("article > a[class^=Listing_link]")]
    return l

links = getCarLinks(soup)

links

['https://www.bilbasen.dk/brugt/bil/mercedes/eqa250/amg-line-5d/6086614',
 'https://www.bilbasen.dk/brugt/bil/tesla/model-y/long-range-awd-5d/6068319',
 'https://www.bilbasen.dk/brugt/bil/vw/t-cross/15-tsi-150-style-dsg-5d/6087844',
 'https://www.bilbasen.dk/brugt/bil/volvo/v40/20-d4-190-momentum-5d/5579843',
 'https://www.bilbasen.dk/brugt/bil/vw/t-cross/10-tsi-110-life-dsg-5d/6043383',
 'https://www.bilbasen.dk/brugt/bil/ford/kuga/15-tdci-120-trend-aut-5d/6006725',
 'https://www.bilbasen.dk/brugt/bil/vw/id3/pro-s-5d/5784873',
 'https://www.bilbasen.dk/brugt/bil/mercedes/a250-e/13-amg-line-aut-5d/6062680',
 'https://www.bilbasen.dk/brugt/bil/citron/c4-cactus/16-bluehdi-100-iconic-5d/6053732',
 'https://www.bilbasen.dk/brugt/bil/mg/4/nordic-edition-5d/6092610',
 'https://www.bilbasen.dk/brugt/bil/volvo/v60/20-t6-recharge-plus-bright-aut-awd-5d/6053726',
 'https://www.bilbasen.dk/brugt/bil/mercedes/cla250-e/13-advantage-amg-coup-aut-4d/5961439',
 'https://www.bilbasen.dk/brugt/bil/merce

In [87]:
# get single car page

def getCarSoup(link):
    page = requests.get(link, headers=headers)
    if page.ok:
        soup = BeautifulSoup(page.text, 'html.parser')
        # print(page)
        return soup
    else:
        print(page.reason)

carPage = getCarSoup(links[0])
# carPage  
print(carPage)

<!DOCTYPE html>
<html lang="en"><head><meta charset="utf-8"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><link href="https://www.bilbasen.dk/favicon.ico" rel="shortcut icon"/><meta content="width=device-width,initial-scale=1" name="viewport"/><meta content="PmzU04GwUWp-nFvo0avXO6pUm6bEFP8VjumHzThcbhE" name="google-site-verification"/><meta content="nositelinkssearchbox" name="google"/><meta content="ie=edge" http-equiv="x-ua-compatible"/><link href="//i.ebayimg.com" rel="preconnect"/><link href="//www.googletagmanager.com" rel="preconnect"/><link href="//www.google-analytics.com" rel="preconnect"/><link href="//www.google.com" rel="preconnect"/><link as="font" crossorigin="anonymous" href="/Content/fonts/GTWalsheimProRegular/GTWalsheimProRegular.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="/Content/fonts/GTWalsheimProMedium/GTWalsheimProMedium.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" href="/Conten

In [94]:
# get car data

def extractCarData(soupTree):

    model = soupTree.select_one('h1[data-e2e="car-make-model-variant"]')['title']
    price = soupTree.select_one('span[data-e2e="car-retail-price"]').text

    detail_div = soupTree.select("div[aria-label=Detaljer] tr")
    # print(detail_div)
    # print(len(detail_div))
    details = {}
    for row in detail_div:
        # print( row.find_all(['th', 'td']) )
        key = row.select_one('th[role=cell][scope=row]').text
        value = row.select_one('td').text
        details[key] = value
        
    description = soupTree.select_one("div[aria-label=beskrivelse] p")
    # print(description)
    # description.p.unwrap()

    
    model_div = soupTree.select("div[aria-label='Generelle modeloplysninger*'] tr")
    # print(model_div)
    # print(len(model_div))
    model = {}
    for row in model_div:
        # print( row.find_all(['th', 'td']) )
        key = row.select_one('th[role=cell][scope=row]').text
        value = row.select_one('td').text
        model[key] = value

    # "Udstyr og tilbehør"
    equipment_div = soupTree.select("div[aria-label='Udstyr og tilbehør'] tr")
    # print(equipment_div)
    # print(len(equipment_div))
    equipment = []
    for row in equipment_div:
        # print( row.find_all(['th', 'td']) )
        d1 = row.select_one('th[role=cell][scope=row]').text
        equipment.append(d1)
        if row.select_one('td'):
            d2 = row.select_one('td').text
            equipment.append(d2)
       

    data = {}
    data['model'] = model
    data['price'] = price
    data['details'] = details
    data['description'] = description
    data['model_details'] = model
    data['equipment'] = equipment
    data['collected'] = datetime.now().isoformat()
    return data



data = extractCarData(carPage)
# data['equipment']
data

{'model': {'Nypris': '-',
  'Kategori': 'Personbil',
  'Type': 'CUV',
  'Bagagerumsstørrelse': '340 liter',
  'Vægt': '2.040 kg',
  'Bredde': '183 cm',
  'Længde': '446 cm',
  'Højde': '162 cm',
  'Lasteevne': '430 kg',
  'Max. trækvægt m/bremse': '750 kg',
  'Trækhjul': 'Forhjulstræk',
  'ABS-bremser': 'Ja',
  'ESP': 'Ja',
  'Airbags': '7',
  'Døre': '5'},
 'price': '394.900 kr.',
 'details': {'Modelår': '2023',
  '1. registrering': '11/2022',
  'Kilometertal': '25.000 km',
  'Drivmiddel': 'El',
  'Rækkevidde': '(WLTP) 492 km',
  'Batterikapacitet': '66,5 kWh',
  'Energiforbrug': '156 Wh/km',
  'Hjemmeopladning AC': '-',
  'Hurtig opladning DC': '-',
  'Opladningstid DC 10-80%': '-',
  'Periodisk afgift': '780 kr. / år',
  'Ydelse': '190 hk/385 nm',
  'Acceleration': '8,6 sek.',
  'Tophastighed': '160 km/t',
  'Trækvægt': '750 kg',
  'Farve': 'Hvid'},
 'description': <p>Komfort/Teknik: Premiumpakke, Førerassistentpakke, Active Distance Assist DISTRONIC, Anhængertræk med ESP® trailer s

In [89]:
# get link to next page

base_url = 'http://bilbasen.dk'

def getNextPage(tree):
    return base_url + tree.select_one("div a[data-e2e=pagination-next]")['href']
    
next = getNextPage(soup)

next

'http://bilbasen.dk/brugt/bil?includeengroscvr=true&includeleasing=false&page=2'

In [90]:
# loop pages